In [1]:
local_path_mnist_train = "/Users/gunnvantsaini/OneDrive/project_codes/content/dl_basics/vision/sony/data/mnist_train.csv"
local_path_mnist_test = "/Users/gunnvantsaini/OneDrive/project_codes/content/dl_basics/vision/sony/data/mnist_test.csv"

In [2]:
import pandas as pd

In [3]:
mnist_train = pd.read_csv(local_path_mnist_train)
mnist_test = pd.read_csv(local_path_mnist_test)

In [4]:
mnist_train.head(2)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Training loop
- Some way of batching the data
- Some abstraction around model
- Some abstraction around weight updates
- Some abstraction around loss functions
- Logging when I am training (print)

### Batched data

In [5]:
from torch.utils.data import Dataset, DataLoader

In [20]:
class Mnist(Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self,idx):
        x = self.X[idx]
        y = self.y[idx]
        return {'X':x,'y':y}

In [15]:
class Students():
    def __init__(self):
        self.names = []
    def add(self,name):
        self.names.append(name)
    def __len__(self):
        return len(self.names)
    def __getitem__(self,idx):
        return self.names[idx]
        

In [16]:
s = Students()

In [17]:
s.add('abc')

In [18]:
len(s)

1

In [19]:
s[0]

'abc'

In [21]:
X = mnist_train.drop('label',axis=1).values
y = mnist_train['label'].values
d_train = Mnist(X,y)

In [22]:
d_train = DataLoader(d_train,batch_size=32)

In [23]:
next(iter(d_train))

{'X': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'y': tensor([1, 0, 1, 4, 0, 0, 7, 3, 5, 3, 8, 9, 1, 3, 3, 1, 2, 0, 7, 5, 8, 6, 2, 0,
         2, 3, 6, 9, 9, 7, 8, 9])}

### Model Abstraction

In [24]:
from torch import nn

In [62]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(784,3)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(3,10)
        self.softmax = nn.Softmax()
    
    def forward(self,X):
        x = self.layer1(X)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.softmax(x)
        return x
    

In [39]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.classifier = nn.Sequential(
            nn.Linear(784,3),nn.ReLU(),nn.Linear(3,10),nn.Softmax()
        )
        
    def forward(self,X):
        x = self.classifier(X)
        return x

In [40]:
m = Model()
m1 = Model1()

In [41]:
import torch
x = torch.randn((1,784))

In [42]:
m(x)

/var/folders/7h/01sjg2sx6tl3y72r2klm24c80000gn/T/ipykernel_24070/2995878503.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


tensor([[0.0966, 0.0657, 0.1099, 0.1501, 0.1297, 0.0999, 0.0583, 0.0665, 0.1027,
         0.1205]], grad_fn=<SoftmaxBackward0>)

In [43]:
m1(x)

/Users/gunnvantsaini/miniforge3/envs/dl/lib/python3.8/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


tensor([[0.0923, 0.1326, 0.0623, 0.1120, 0.1450, 0.1400, 0.0553, 0.0922, 0.1113,
         0.0571]], grad_fn=<SoftmaxBackward0>)

### Abstraction around weight updates

In [44]:
import torch.optim as opt

In [45]:
o = opt.SGD(m.parameters(),lr=0.01)

### Abstraction around loss functions

In [47]:
ce = nn.CrossEntropyLoss()

### Training Loop 

In [65]:
X = mnist_train.drop('label',axis=1).values/255.0
y = mnist_train['label'].values
d_train = Mnist(X,y)
d_train = DataLoader(d_train,batch_size=8)
m = Model()
o = opt.SGD(m.parameters(),lr=0.01)
epochs = 10
for i in range(epochs):
    Losses = []
    Accuracies = []
    for j,batch in enumerate(d_train):
        x = batch['X'].float()
        y = batch['y']
        p = m(x)
        acc = (p.argmax(axis=1)==y).float().mean().item()
        Accuracies.append(acc)
        loss = ce(p,y)
        Losses.append(loss.item())
        loss.backward()
        o.step()
        o.zero_grad()
    l = sum(Losses)/len(Losses)    
    a = sum(Accuracies)/len(Accuracies)
    print(f'Loss:{l}, acc:{a}, epoch:{i}')

/var/folders/7h/01sjg2sx6tl3y72r2klm24c80000gn/T/ipykernel_24070/2995878503.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


Loss:2.192112795738947, acc:0.2609761904761905, epoch:0
Loss:2.0375873780931744, acc:0.44895238095238094, epoch:1
Loss:1.9459630636260623, acc:0.6071666666666666, epoch:2
Loss:1.848797149135953, acc:0.6755952380952381, epoch:3
Loss:1.8068892265955607, acc:0.6905238095238095, epoch:4
Loss:1.7871411216372535, acc:0.6973809523809524, epoch:5
Loss:1.7761115570295425, acc:0.7028809523809524, epoch:6
Loss:1.7690182934715635, acc:0.7063809523809523, epoch:7
Loss:1.7639548227673485, acc:0.7088809523809524, epoch:8
Loss:1.7600223141851878, acc:0.7103571428571429, epoch:9


### Prediction Loop
- Prediction data in a form that I can use my model on
- Don't want to track the gradients

In [66]:
mnist_test.head(2)

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
X = mnist_test.values/255.0

In [68]:
class Mnist_predict(Dataset):
    def __init__(self,X):
        self.X = X
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self,idx):
        x = self.X[idx]
        return {'X':x}

In [69]:
predict_data = Mnist_predict(X)
predict_data = DataLoader(predict_data,batch_size=1)

In [71]:
predictions = []
for batch in predict_data:
    x = batch['X'].float()
    with torch.no_grad():
        p=m(x)
        predictions.append(p.argmax(axis=1).item())

/var/folders/7h/01sjg2sx6tl3y72r2klm24c80000gn/T/ipykernel_24070/2995878503.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


In [73]:
predictions[0:10]

[2, 0, 9, 4, 2, 7, 0, 2, 0, 0]

### Convolutional Neural Networks: Writting an architecture

$n_{out} = \frac{n_{in}+2p-k}{s}+1$

In [97]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6,kernel_size=5,stride=1,padding=0)
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
        self.conv2 = nn.Conv2d(in_channels = 6,out_channels = 16,kernel_size=5,stride=1,padding=0)
        self.linear1 = nn.Linear(4*4*16,120)### change this to handle mnist data
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(120,84)
        self.linear3 = nn.Linear(84,10)
        self.softmax = nn.Softmax(dim=1)
        self.flatten = nn.Flatten()
    
    def forward(self,X):
        x = self.conv1(X)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.relu(x) ## batch,5,5,16
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = self.softmax(x)
        return x
        

In [98]:
cnn = CNN()## use on mnist data (28*28)

In [99]:
img = torch.randn((1,1,28,28))

In [100]:
cnn(img)

tensor([[0.1012, 0.0986, 0.0915, 0.0997, 0.0959, 0.0867, 0.1065, 0.1090, 0.1061,
         0.1049]], grad_fn=<SoftmaxBackward0>)

In [96]:
256/16

16.0

In [106]:
class Mnist_CNN(Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self,idx):
        x = self.X[idx]
        y = self.y[idx]
        return {'X':x.reshape((1,28,28)),'y':y}

In [108]:
X = mnist_train.drop('label',axis=1).values/255.0
y = mnist_train['label'].values
d_train = Mnist_CNN(X,y)
d_train = DataLoader(d_train,batch_size=32)
mod = CNN()
ce = nn.CrossEntropyLoss()
o = opt.Adam(mod.parameters(),lr=0.001)
epochs = 10
for i in range(epochs):
    Losses = []
    Accuracies = []
    for j,batch in enumerate(d_train):
        x = batch['X'].float()
        y = batch['y']
        p = mod(x)
        acc = (p.argmax(axis=1)==y).float().mean().item()
        Accuracies.append(acc)
        loss = ce(p,y)
        Losses.append(loss.item())
        loss.backward()
        o.step()
        o.zero_grad()
    l = sum(Losses)/len(Losses)    
    a = sum(Accuracies)/len(Accuracies)
    print(f'Loss:{l}, acc:{a}, epoch:{i}')

Loss:1.6320313571158567, acc:0.8343726199543031, epoch:0
Loss:1.502194032447536, acc:0.9600390327494288, epoch:1
Loss:1.4927641727138092, acc:0.96875, epoch:2
Loss:1.4859093868904956, acc:0.9758187357197258, epoch:3
Loss:1.483908484296225, acc:0.9775561690784463, epoch:4
Loss:1.481959877627911, acc:0.9794126047220106, epoch:5
Loss:1.4807682477692476, acc:0.980436024371668, epoch:6
Loss:1.4795370581308032, acc:0.9815784463061691, epoch:7
Loss:1.4780893658102694, acc:0.9830778750952018, epoch:8
Loss:1.4783071876026161, acc:0.9828874714394517, epoch:9


### Transfer Learning
- Pre-trained models

In [110]:
from torchvision import models

In [113]:
m1 = models.vgg16(pretrained=True)

In [114]:
path = "../codes/imgs/elephant.jpg"
import PIL

In [115]:
img = PIL.Image.open(path)

In [118]:
from torchvision import transforms

In [124]:
size = 256
image_transforms=transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
                                transforms.Resize(size)])

In [125]:
img = image_transforms(img)

In [126]:
img.shape

torch.Size([3, 256, 426])

In [128]:
img=img.unsqueeze(0)

In [130]:
m1(img).argmax(axis=1)

tensor([386])

In [133]:
import pandas as pd
path_labels = "../data/vgg16_model_labels.csv"
lookup = pd.read_csv(path_labels,header=None)

In [142]:
lookup[lookup[0]==386][1]

386    African elephant, Loxodonta africana
Name: 1, dtype: object

### How to work with data (transfer learning)

In [143]:
path = "/Users/gunnvantsaini/Documents/Data/Work/ML Course/Module 5 Neural Networks/Data/waffle_pancakes/train"
from torchvision import datasets

In [144]:
size = 150
image_transforms=transforms.Compose([transforms.Resize((size,size)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
train=datasets.ImageFolder(path,image_transforms)

In [148]:
train_batched = DataLoader(train,batch_size=10)

In [150]:
train.classes

['pancakes', 'waffles']

In [151]:
train.class_to_idx

{'pancakes': 0, 'waffles': 1}

In [157]:
m1.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [159]:
for p in m1.parameters():
    p.requires_grad=False

In [161]:
m1.classifier = nn.Sequential(nn.Linear(7*7*512,20),nn.ReLU(),nn.Linear(20,10),nn.ReLU(),nn.Linear(10,2),nn.Softmax())

In [163]:
for p in m1.parameters():
    print(p.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True


In [166]:
ce = nn.CrossEntropyLoss()
o = opt.SGD(m1.parameters(),lr=0.01)
epochs = 1
for i in range(epochs):
    Losses = []
    Accuracies = []
    for j,batch in enumerate(train_batched):
        x,y = batch
        p = m1(x)
        acc = (p.argmax(axis=1)==y).float().mean().item()
        Accuracies.append(acc)
        loss = ce(p,y)
        Losses.append(loss.item())
        loss.backward()
        o.step()
        o.zero_grad()
    l = sum(Losses)/len(Losses)    
    a = sum(Accuracies)/len(Accuracies)
    print(f'Loss:{l}, acc:{a}, epoch:{i}')

/Users/gunnvantsaini/miniforge3/envs/dl/lib/python3.8/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Loss:0.5064109573238774, acc:0.8210526319710831, epoch:0
